In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = saw
band = [11]
commitnumber = de5d43e
molecule = {'o3': 'atmpro'}
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o o3 band9
  atmpro = mls
  band = 9
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 2e-20
  molecule = o3
  ng_adju = [0, 0]
  ng_refs = [2, 5]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (50, 250)]
  tsfc = 294
  vmax = 3000
  vmin = 1900
  w_diffuse = [(1.55, 1.55), (1.55, 1.55, 1.55, 1.55, 1.8)]
  wgt = [(0.3, 0.4), (0.5, 0.6, 0.7, 0.85, 0.9)]


# Cooling Rate by Layer

CRD               CLIRAD `coolr_bands`              
            layer        coolrg                layer        coolrg
pressure                                                          
0.000312      1.0 -5.372597e-05                  NaN           NaN
0.000312      NaN           NaN                  1.0 -4.823381e-05
0.000750      2.0 -5.921783e-05                  2.0 -8.232329e-05
0.001052      3.0 -6.640717e-05                  3.0 -5.208771e-05
0.001476      4.0 -6.726871e-05                  4.0 -4.941980e-05
0.002070      5.0 -6.005722e-05                  5.0 -4.189689e-05
0.002904      6.0 -4.257966e-05                  6.0 -2.684817e-05
0.004074      7.0 -1.213240e-05                  7.0 -2.859706e-06
0.005714      8.0  3.451123e-05                  8.0  3.266592e-05
0.008015      9.0  1.012449e-04                  9.0  8.269105e-05
0.011243     10.0  1.926961e-04                 10.0  1.510018e-04
0.015771     11.0  3.147070e-04                 11.0  2.417455e-04
0.022122     12.0  4.737022e-04                 12.0  3.598487e-04
0.031031     13.0  6.766232e-04                 13.0  5.117008e-04
0.043528     14.0  9.869767e-04                 14.0  7.437226e-04
0.061057     15.0  1.458650e-03                 15.0  1.095401e-03
0.085645     16.0  2.053613e-03                 16.0  1.539210e-03
0.120136     17.0  2.770155e-03                 17.0  2.076890e-03
0.168516     18.0  3.623520e-03                 18.0  2.720136e-03
0.236378     19.0  4.629553e-03                 19.0  3.482268e-03
0.331549     20.0  5.805584e-03                 20.0  4.377204e-03
0.465100     21.0  7.092450e-03                 21.0  5.363414e-03
0.652400     22.0  7.705676e-03                 22.0  5.849211e-03
0.915100     23.0  6.731779e-03                 23.0  5.148924e-03
1.283650     24.0  5.153469e-03                 24.0  3.992583e-03
1.800600     25.0  3.519644e-03                 25.0  2.763514e-03
2.525700     26.0  1.731645e-03                 26.0  1.352453e-03
3.542800     27.0  2.251784e-04                 27.0  1.013232e-04
4.969550     28.0 -6.902491e-04                 28.0 -7.119592e-04
6.970850     29.0 -1.065652e-03                 29.0 -1.085055e-03
9.778100     30.0 -1.223068e-03                 30.0 -1.259409e-03
13.715850    31.0 -1.398161e-03                 31.0 -1.456280e-03
19.239350    32.0 -1.630786e-03                 32.0 -1.714780e-03
26.987250    33.0 -1.839465e-03                 33.0 -1.962590e-03
37.855300    34.0 -1.930795e-03                 34.0 -2.086318e-03
53.100050    35.0 -1.920361e-03                 35.0 -2.070225e-03
73.887500    36.0 -1.809605e-03                 36.0 -1.903798e-03
97.662500    37.0 -1.533815e-03                 37.0 -1.549854e-03
121.437500   38.0 -1.224882e-03                 38.0 -1.185589e-03
145.212500   39.0 -1.004489e-03                 39.0 -9.346214e-04
168.987500   40.0 -8.556182e-04                 40.0 -7.682579e-04
192.762500   41.0 -6.728651e-04                 41.0 -5.854617e-04
216.537500   42.0 -5.026385e-04                 42.0 -4.261097e-04
240.312500   43.0 -3.838611e-04                 43.0 -3.186680e-04
264.087500   44.0 -2.864587e-04                 44.0 -2.338612e-04
287.862500   45.0 -2.098350e-04                 45.0 -1.691184e-04
311.637500   46.0 -1.475549e-04                 46.0 -1.178611e-04
335.412500   47.0 -1.030715e-04                 47.0 -8.185532e-05
359.187500   48.0 -7.822285e-05                 48.0 -6.182593e-05
382.962500   49.0 -6.050444e-05                 49.0 -4.758373e-05
406.737500   50.0 -4.532636e-05                 50.0 -3.545382e-05
430.512500   51.0 -3.236019e-05                 51.0 -2.513869e-05
454.287500   52.0 -2.304723e-05                 52.0 -1.773714e-05
478.062500   53.0 -1.668757e-05                 53.0 -1.263745e-05
501.837500   54.0 -1.097308e-05                 54.0 -8.051688e-06
525.612500   55.0 -5.609332e-06                 55.0 -3.749180e-06
549.387500   56.0 -5.931302e-07                 56.0  2.81003

# Fluxes by Layer

CRD                            CLIRAD            \
                        flug          fldg     fnetg      flug      fldg   
pressure     level                                                         
0.000000e+00 1     -1.425259  0.000000e+00 -1.425259       NaN       NaN   
1.000000e-08 1           NaN           NaN       NaN -1.425814  0.000002   
6.244000e-04 2     -1.425259  1.227340e-09 -1.425259 -1.425814  0.000002   
8.759000e-04 3     -1.425259  1.854404e-09 -1.425259 -1.425814  0.000002   
1.228600e-03 4     -1.425259  3.197797e-09 -1.425259 -1.425814  0.000002   
1.723400e-03 5     -1.425259  5.947331e-09 -1.425259 -1.425814  0.000002   
2.417400e-03 6     -1.425259  1.138825e-08 -1.425259 -1.425814  0.000002   
3.390900e-03 7     -1.425259  2.189569e-08 -1.425259 -1.425814  0.000002   
4.756500e-03 8     -1.425259  4.180017e-08 -1.425259 -1.425814  0.000002   
6.672000e-03 9     -1.425259  7.892069e-08 -1.425259 -1.425814  0.000002   
9.358900e-03 10    -1.425259  1.472777e-07 -1.425259 -1.425814  0.000002   
1.312780e-02 11    -1.425259  2.717602e-07 -1.425259 -1.425814  0.000003   
1.841450e-02 12    -1.425259  4.965479e-07 -1.425259 -1.425814  0.000003   
2.583020e-02 13    -1.425259  8.989709e-07 -1.425258 -1.425814  0.000003   
3.623230e-02 14    -1.425259  1.613421e-06 -1.425257 -1.425814  0.000004   
5.082340e-02 15    -1.425259  2.958884e-06 -1.425256 -1.425814  0.000005   
7.129060e-02 16    -1.425258  5.604600e-06 -1.425252 -1.425813  0.000007   
1.000000e-01 17    -1.425256  1.062671e-05 -1.425245 -1.425812  0.000010   
1.402710e-01 18    -1.425252  1.979908e-05 -1.425232 -1.425809  0.000017   
1.967600e-01 19    -1.425244  3.609571e-05 -1.425208 -1.425803  0.000029   
2.759970e-01 20    -1.425229  6.443605e-05 -1.425164 -1.425791  0.000050   
3.871000e-01 21    -1.425201  1.128406e-04 -1.425088 -1.425770  0.000087   
5.431000e-01 22    -1.425150  1.936154e-04 -1.424957 -1.425732  0.000148   
7.617000e-01 23    -1.425074  3.164708e-04 -1.424757 -1.425675  0.000243   
1.068500e+00 24    -1.424986  4.734847e-04 -1.424513 -1.425612  0.000367   
1.498800e+00 25    -1.424906  6.558234e-04 -1.424250 -1.425559  0.000517   
2.102400e+00 26    -1.424854  8.557045e-04 -1.423998 -1.425536  0.000692   
2.949000e+00 27    -1.424867  1.042577e-03 -1.423825 -1.425577  0.000868   
4.136600e+00 28    -1.424983  1.190389e-03 -1.423793 -1.425716  0.001022   
5.802500e+00 29    -1.425230  1.300390e-03 -1.423929 -1.425982  0.001147   
8.139200e+00 30    -1.425616  1.391476e-03 -1.424224 -1.426390  0.001255   
1.141700e+01 31    -1.426177  1.478184e-03 -1.424699 -1.426980  0.001355   
1.601470e+01 32    -1.427034  1.573424e-03 -1.425461 -1.427880  0.001462   
2.246400e+01 33    -1.428403  1.696530e-03 -1.426706 -1.429321  0.001594   
3.151050e+01 34    -1.430577  1.899608e-03 -1.428678 -1.431626  0.001795   
4.420010e+01 35    -1.433827  2.246823e-03 -1.431580 -1.435092  0.002125   
6.200000e+01 36    -1.438382  2.753155e-03 -1.435629 -1.439936  0.002603   
8.577500e+01 37    -1.444099  3.373111e-03 -1.440725 -1.445892  0.003197   
1.095500e+02 38    -1.448915  3.869283e-03 -1.445045 -1.450741  0.003680   
1.333250e+02 39    -1.452727  4.232099e-03 -1.448495 -1.454442  0.004042   
1.571000e+02 40    -1.455827  4.503494e-03 -1.451324 -1.457352  0.004320   
1.808750e+02 41    -1.458453  4.719517e-03 -1.453734 -1.459740  0.004544   
2.046500e+02 42    -1.460510  4.881091e-03 -1.455628 -1.461559  0.004714   
2.284250e+02 43    -1.462042  4.997735e-03 -1.457044 -1.462884  0.004838   
2.522000e+02 44    -1.463210  5.084835e-03 -1.458125 -1.463875  0.004931   
2.759750e+02 45    -1.464081  5.148944e-03 -1.458932 -1.464602  0.005000   
2.997500e+02 46    -1.464719  5.196523e-03 -1.459523 -1.465129  0.005051   
3.235250e+02 47    -1.465174  5.235576e-03 -1.459938 -1.465502  0.005092   
3.473000e+02 48    -1.465502  5.273025e-03 -1.460229 -1.465769  0.005129   
3.710750e+02 49    -1.465762  5.313400e-03 -1.460449 -1.465982  0.005167 

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')